In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from datetime import datetime
from IPython.display import display, Markdown
from pathvalidate import sanitize_filename

import utils
from prompts import Job_Post, Resume_Builder

In [ ]:
## Inputs
my_files_dir = "my_applications" # location for all job and resume files
job_file = "job.txt" # filename with job post text. The entire job post can be pasted in this file, as is.
raw_resume_file = "resume_raw.yaml" # filename for raw resume yaml. See example in repo for instructions.

# open_ai model. gpt-4 is not publicly available yet and can be 20-30 times costlier than the default model gpt-3.5-turbo
openai_model_name = "gpt-4"
openai_model_name = "gpt-3.5-turbo"

In [ ]:
# Step 1 - Read and parse job posting
job_post = Job_Post(
    utils.read_jobfile(os.path.join(my_files_dir, job_file)),
    temperature=0.3,
    model_name=openai_model_name,
)
parsed_job = job_post.parse_job_post()

company_name = parsed_job["company"]
job_title = parsed_job["job_title"]
today_date = datetime.today().strftime("%Y%m%d")
job_filename = os.path.join(my_files_dir, sanitize_filename(f"{today_date}__{company_name}__{job_title}"))
utils.write_yaml(parsed_job, filename=f"{job_filename}.job")

In [ ]:
# Step 2 - read raw resume and create Resume builder object
my_resume = Resume_Builder(
    raw_resume=utils.read_yaml(filename=os.path.join(my_files_dir, raw_resume_file)),
    job_post=job_post,
    temperature=0.2,
    model_name=openai_model_name,
)
projects = my_resume.projects_raw
experiences = my_resume.experiences_raw
skills = my_resume.skills_raw

In [ ]:
# Step 3 - Rephrase unedited experiences
experiences = my_resume.rewrite_unedited_experiences(verbose=False)
utils.write_yaml(dict(experiences=experiences))

In [ ]:
# Review the generated output in previous cell.
# If any updates are needed, copy the cell output below between the triple quotes
# Set value to """" """" if no edits are needed
edits = """ """

edits = edits.strip()
if edits:
    edits1 = utils.read_yaml(edits)
    if "experiences" in edits1:
        experiences = edits1["experiences"]
    if "projects" in edits1:
        projects = edits1["projects"]
    if "skills" in edits1:
        skills = edits1["skills"]
    if "summary" in edits1:
        summary = edits1["summary"]

In [ ]:
# Step 4 - Rephrase projects
my_resume.experiences = experiences

projects = my_resume.rewrite_section(section=projects, verbose=False)
utils.write_yaml(dict(projects=projects))

In [ ]:
# Review the generated output in previous cell.
# If any updates are needed, copy the cell output below between the triple quotes
# Set value to """" """" if no edits are needed
edits = """ """

edits = edits.strip()
if edits:
    edits2 = utils.read_yaml(edits)
    if "experiences" in edits2:
        experiences = edits2["experiences"]
    if "projects" in edits2:
        projects = edits2["projects"]
    if "skills" in edits2:
        skills = edits2["skills"]
    if "summary" in edits1:
        summary = edits2["summary"]

In [ ]:
# Step 5 - Extract skills
# This will match the required skills from the job post with your resume sections
# Outputs a combined list of skills extracted from the job post and included in the raw resume
my_resume.experiences = experiences
my_resume.projects = projects

skills = my_resume.extract_skills(verbose=False)
utils.write_yaml(dict(skills=skills))

In [ ]:
# Review the generated output in previous cell.
# If any updates are needed, copy the cell output below between the triple quotes
# Set value to """" """" if no edits are needed
edits = """ """

edits = edits.strip()
if edits:
    edits3 = utils.read_yaml(edits)
    if "experiences" in edits3:
        experiences = edits3["experiences"]
    if "projects" in edits3:
        projects = edits3["projects"]
    if "skills" in edits3:
        skills = edits3["skills"]
    if "summary" in edits3:
        summary = edits3["summary"]

In [ ]:
# Step 6 - Create a resume summary
my_resume.experiences = experiences
my_resume.skills = skills
my_resume.projects = projects

summary = my_resume.create_summary(verbose=False)
utils.write_yaml(dict(summary=summary))

In [ ]:
# Review the generated output in previous cell.
# If any updates are needed, copy the cell output below between the triple quotes
# Set value to """" """" if no edits are needed.
edits = """ """

edits = edits.strip()
if edits:
    edits4 = utils.read_yaml(edits)
    if "experiences" in edits4:
        experiences = edits4["experiences"]
    if "projects" in edits4:
        projects = edits4["projects"]
    if "skills" in edits4:
        skills = edits4["skills"]
    if "summary" in edits4:
        summary = edits4["summary"]

In [ ]:
# Step 7 - Generate final resume yaml for review
my_resume.summary = summary
my_resume.experiences = experiences
my_resume.projects = projects
my_resume.skills = skills

today_date = datetime.today().strftime("%Y%m%d")
resume_filename = os.path.join(
    my_files_dir, sanitize_filename(f"{today_date}__{company_name}__{job_title}")
)
resume_final = my_resume.finalize()
print(f"#filename: {resume_filename}.yaml\n")
utils.write_yaml(resume_final, filename=f"{resume_filename}.yaml")

In [ ]:
# Step 8 - Identify resume improvements
final_resume = utils.read_yaml(filename=f"{resume_filename}.yaml")
my_resume.experiences = final_resume['experiences']
my_resume.skills = final_resume['skills']
my_resume.projects = final_resume['projects']

improvements = my_resume.suggest_improvements(verbose=False)
utils.write_yaml(dict(improvements=improvements))

In [ ]:
# Update the yaml file directly as needed based on suggested improvements.
# You can re-run the previous and the next cells as many times after editing the yaml file.

In [ ]:
# Step 9 - Generate pdf from yaml
# Most common errors during pdf generation occur due to special characters. Escape them with backslashes in the yaml, e.g. $ -> \$
pdf_file = utils.generate_pdf(yaml_file=f"{resume_filename}.yaml")
display(Markdown((f"[{pdf_file}](<{pdf_file}>)")))